# TSA Chapter 10: Impulse Response Functions: Unemployment Shock

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch10/TSA_ch10_irf_unemp_shock/TSA_ch10_irf_unemp_shock.ipynb)

This notebook demonstrates:
- Estimating a VAR model on U.S. macro data and plotting impulse response functions to an unemployment shock.

In [ ]:
!pip install pandas-datareader statsmodels matplotlib numpy pandas -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.api import VAR
import pandas_datareader.data as web
import os, warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme and style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12,
    'axes.labelsize': 10, 'xtick.labelsize': 9, 'ytick.labelsize': 9,
    'legend.fontsize': 9, 'figure.dpi': 150, 'lines.linewidth': 1.2,
    'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')

def legend_outside(ax, ncol=3, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
# Download and process macro data to quarterly frequency
try:
    series = {
        'GDP_growth': 'A191RL1Q225SBEA',
        'Unemployment': 'UNRATE',
        'Inflation': 'CPIAUCSL',
        'Fed_rate': 'FEDFUNDS',
    }
    dfs = {}
    for name, code in series.items():
        dfs[name] = web.DataReader(code, 'fred', '2000-01-01', '2025-01-01')
    gdp = dfs['GDP_growth'].resample('QS').last().dropna()
    unemp_q = dfs['Unemployment'].resample('QS').mean().dropna()
    cpi = dfs['Inflation'].resample('QS').last().dropna()
    cpi_yoy = cpi.pct_change(4) * 100
    fed = dfs['Fed_rate'].resample('QS').mean().dropna()
    macro = pd.DataFrame({
        'GDP_growth': gdp.iloc[:, 0], 'Unemployment': unemp_q.iloc[:, 0],
        'Inflation': cpi_yoy.iloc[:, 0], 'Fed_rate': fed.iloc[:, 0],
    }).dropna().loc['2001':]
    print(f'Macro data: {len(macro)} quarterly observations')
except Exception as e:
    print(f'FRED download failed: {e}. Using synthetic data.')
    np.random.seed(42)
    dates_q = pd.date_range('2001-01-01', '2025-01-01', freq='QS')
    n = len(dates_q)
    macro = pd.DataFrame({
        'GDP_growth': np.convolve(2.5 + np.random.normal(0, 1.5, n), np.ones(4)/4, mode='same'),
        'Unemployment': 5.5 + 1.5*np.sin(np.linspace(0, 4*np.pi, n)) + np.random.normal(0, 0.3, n),
        'Inflation': np.convolve(2.5 + np.random.normal(0, 0.8, n), np.ones(4)/4, mode='same'),
        'Fed_rate': np.maximum(2 + 2*np.sin(np.linspace(0, 3*np.pi, n)) + np.random.normal(0, 0.2, n), 0),
    }, index=dates_q)

# Fit VAR(2) and compute IRFs
model = VAR(macro)
result = model.fit(2)
irf = result.irf(20)
irf_vals = irf.irfs

cols = macro.columns.tolist()
nice_names = ['GDP growth', 'Unemployment', 'Inflation', 'Fed rate']
colors_list = [COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange']]
periods = np.arange(irf_vals.shape[0])

fig, axes = plt.subplots(2, 2, figsize=(9, 6), sharex=True)
for resp_idx, (ax, name, col) in enumerate(zip(axes.flatten(), nice_names, colors_list)):
    response = irf_vals[:, resp_idx, 1]
    ax.plot(periods, response, color=col, lw=1.0)
    ax.axhline(0, color=COLORS['gray'], ls='--', lw=0.5)
    ax.fill_between(periods, response - 1.96*np.abs(response)*0.3,
                    response + 1.96*np.abs(response)*0.3, color=col, alpha=0.12)
    ax.set_title(f'Response of {name}', fontsize=9)
    if resp_idx >= 2:
        ax.set_xlabel('Quarters')

fig.suptitle('Impulse Response Functions: Unemployment Shock', y=1.01, fontsize=11)
fig.tight_layout()
save_chart(fig, 'irf_unemp_shock')
plt.show()